In [1]:
import datetime
import threading
import pandas as pd
from web3 import Web3
from pprint import pprint as pp
# 여러개의 CSV를 처리하기 위해서 사용
from multiprocessing import Process, Pool
# 한 CSV에서 탐색하기 위해서 사용
from threading import Thread, Lock
from secrete import *

base_URL = "https://mainnet.infura.io/v3/"
base_Input_PATH = "../input/"
base_Output_PATH = "../output/"
MAX_Chunk_Number = 362
MAX_Quiry = 100000
MAX_Thread_Quiry = 20000

INFURA_URL_Limit_List = {key: 80000 for key in INFURA_URL_List}
used_urls = set()

In [2]:
def is_eoa(w3, address, lock):
    try:
        checksum_address = w3.to_checksum_address(address)
        uri = (w3.provider.endpoint_uri).replace(base_URL,"")
        with lock:
            INFURA_URL_Limit_List[uri] += 1
        return w3.eth.get_code(checksum_address) == b''
    except Exception as e:
        Exception(f"{uri} key is expired")
        

def find_available_url(lock):
    with lock:
        for url in INFURA_URL_List:
            if url not in used_urls and MAX_Quiry - INFURA_URL_Limit_List[url] >= MAX_Thread_Quiry:
                used_urls.add(url)
                return url

        raise Exception("All API Keys expired or reached limit")


def work_thread(ith, chunk_df, return_df, lock):
    url_INFURA = find_available_url(lock)
    w3 = Web3(Web3.HTTPProvider(base_URL + url_INFURA))
    print(f"{threading.current_thread().name} Start")
    print(f"Using API-Key:{url_INFURA}")

    # Convert the addresses to checksum format after handling NaN values
    chunk_df = chunk_df.copy()
    chunk_df['from_address'] = chunk_df['from_address'].apply(w3.to_checksum_address)
    chunk_df['to_address'] = chunk_df['to_address'].apply(w3.to_checksum_address)

    try:
        eoa_df = chunk_df[
            (chunk_df['from_address'].apply(lambda x: is_eoa(w3, x, lock))) & 
            (chunk_df['to_address'].apply(lambda x: is_eoa(w3, x, lock)))
        ]
        return_df.insert(ith, eoa_df)
        used_urls.remove(url_INFURA)
    except Exception as e:
        raise Exception(e)
            
def refine_INFURA(nProcess: str, file_Name: str):
    print(f"Process{nProcess} start reading {file_Name}")
    # 1 process 4 INFURA_URL
    file_Path = base_Input_PATH+file_Name
    chunk_df = pd.read_csv(file_Path)
    lock = Lock()

    # Drop rows with NaN or empty values in 'from_address' or 'to_address'
    chunk_df = chunk_df.dropna(subset=['from_address', 'to_address'])
    thread_List = []
    output_df_list = []

    addition_Count = 10000
    start = 0
    lock = Lock()

    for nThread in range(4):
        end = start + addition_Count

        thread_List.append(Thread(target=work_thread, args=(nThread, chunk_df.loc[start: end], output_df_list, lock)))
        start += addition_Count

        try:
            thread_List[nThread].start()
        except Exception as e:
            raise Exception(f"All API Key expired\nLast FileName: {file_Name}")

    print(f"All Thread Started at {datetime.datetime.now()}")

    for thread in thread_List:
        thread.join()

    print("All Thread Complete")
    eoa_df = output_df_list[0]
    for i in range(1, 4):
        eoa_df = pd.concat([eoa_df, output_df_list[i]])

    # Save the filtered dataframe to result_1.csv
    now = datetime.datetime.now().strftime("%Y.%m.%d")
    eoa_df.to_csv(f"{base_Output_PATH}reulst_{file_Name}({now}).csv", index=False)

    print(f"Process{nProcess} Done\nOutput: reulst_{file_Name}({now}).csv")
    pp(INFURA_URL_Limit_List)

In [3]:
if __name__ == "__main__":
    try:
        pp(INFURA_URL_Limit_List)
        for i in range(156, 157):
            file_name = "chunk_"+str(i)+".csv"
            nProcess = 1
            refine_INFURA(nProcess, file_name)
    except Exception as e:
        print(e)

{'20567e8f9e8344f5a336e57643058701': 80000,
 '2ca7a7a7d67047b383efad93ff1df83e': 80000,
 '42126694461a48b6ae41c1111853b0ec': 80000,
 '5cb46bbaa9394d9c99867531b15a9e2d': 80000,
 '7ac3c6a3dc4f4064a521b860af0b6042': 80000,
 '7e91e2ed8b66439ebe59fc9ca39c3c79': 80000,
 '8089eeaa89444e389768e1425eeb9668': 80000,
 '9218b856639b4f3ab4c9f6e915eb2920': 80000,
 'af52805d7f8a42b981b5b35d8128a415': 80000,
 'b116074cf1824f7b9f8c391487993689': 80000,
 'd141e645c6f64658b6f3877953353970': 80000,
 'd5e3d9b5a2344380aaf28566c4492b2a': 80000,
 'e2e9af8bb5734d9aae01721a2846f4f3': 80000,
 'e7494fa62d2c4ef38bef2afafb916814': 80000,
 'e84f01d708834224b9161637bd5e885c': 80000,
 'f12e853175ef47898b1088be5805688b': 80000}
Process1 start reading chunk_157.csv
Thread-5 (work_thread) Start
Using API-Key:20567e8f9e8344f5a336e57643058701
Thread-6 (work_thread) Start
Using API-Key:5cb46bbaa9394d9c99867531b15a9e2d
Thread-7 (work_thread) Start
Using API-Key:af52805d7f8a42b981b5b35d8128a415
Thread-8 (work_thread) Start
Us